### 1.2.2 レイヤとしてのクラス化と順伝播の実装

- Python版ではレイヤをclassで定義していたが，Juliaではclassの概念が存在しない．
  - モデルをコンストラクタで定義し，それにメソッドを作用させていくように考える．
- そのため，レイヤに持たせていたメソッド`forward()`と`backward()`，インスタンス変数`params`と`grads`を独立させる．
  - メソッドは`function`で定義し，インスタンス変数は`struct`に持たせる．
  - Juliaの多重ディスパッチの機構を利用し，同じ名前のメソッドでも型によって異なる処理を呼び出す．
    - レイヤを型として定義する．

In [1]:
# 抽象型の定義
abstract type Layer end

In [2]:
# Sigmoidレイヤの定義
mutable struct Sigmoid <: Layer
    params
    # コンストラクタメソッド
    Sigmoid() = new()
end

In [3]:
# Affineレイヤの定義
mutable struct Affine <: Layer
    params
    W
    b
    function Affine(W, b)
        params = [W, b]
        new(params)
    end
end

In [4]:
# Python版のSigmoidクラスのforward()
function forward(x, self::Sigmoid)
    return 1.0 ./ (1.0 .+ exp.(-x))
end

forward (generic function with 1 method)

In [5]:
# Python版のAffineクラスのforward()
function forward(x, self::Affine)
    W, b = self.params
    return x * W .+ reshape(b, 1, :) # bを1x4 Arrayにreshape
end

forward (generic function with 2 methods)

In [6]:
# TwoLayerNetの定義
mutable struct TwoLayerNet
    I # input_size
    H # hidden_size
    O # output_size
    W1
    b1
    W2
    b2
    layers
    
    # コンストラクタメソッド
    function TwoLayerNet(I, H, O)
        # 重みとバイアスの初期化
        W1 = randn(I, H)
        b1 = randn(H)
        W2 = randn(H, O)
        b2 = randn(O)
        
        # レイヤの生成
        layers = [
            Affine(W1, b1),
            Sigmoid(),
            Affine(W2, b2)
        ]
        new(I, H, O, W1, b1, W2, b2, layers)
    end
end

In [7]:
# Python版のTwoLayerNetのpredict()
function predict(x, model::TwoLayerNet)
    for layer in model.layers
        x = forward(x, layer)
    end
    return x
end

predict (generic function with 1 method)

In [8]:
x = randn(10, 2)
model = TwoLayerNet(2, 4, 3)
s = predict(x, model)

10×3 Array{Float64,2}:
 0.625462   0.0877237   1.96784
 0.399283  -0.218909    1.18103
 0.651246  -0.072969    2.55017
 0.533274  -0.162174    1.50475
 1.13087   -0.630072    1.56645
 0.715964  -0.306506    1.45
 1.09445   -0.741227    1.47343
 0.559469  -0.092444    1.76157
 0.962665  -0.700011    0.751202
 0.507741  -0.00670891  1.53918